In [2]:
! pip install pygame


In [3]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer

2024-10-25 00:58:43.519815: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
model = load_model('/Users/air/Desktop/Prepared Data/models/model.keras')

In [2]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer
import time

# Initialize the pygame mixer
mixer.init()

# Load the alarm sound
sound = mixer.Sound('/Users/air/Downloads/Driver-Drowsiness-Detection-using-Deep-Learning-main/Driver-Drowsiness-Detection-using-Deep-Learning-main/alarm.wav')

# Load the face and eye cascade classifiers
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Load the pre-trained model
model = load_model('/Users/air/Desktop/Prepared Data/models/model.keras')

# Open the webcam with a smaller frame size to increase speed
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

Score = 0
thresh_open = 0.90  # Adjusted threshold for open eyes
thresh_closed = 0.30  # Adjusted threshold for closed eyes
head_movement_timer = 0  # Timer for no head movement
no_movement_duration = 15  # Duration in seconds for highway hypnosis alert
last_head_position = None  # Store the previous head position

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=3)

    # Draw a rectangle at the bottom of the frame for text
    height, width = frame.shape[0:2]
    cv2.rectangle(frame, (0, height - 30), (200, height), (0, 0, 0), thickness=cv2.FILLED)

    # Process each face
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        face_gray = gray[y:y+h, x:x+w]
        face_color = frame[y:y+h, x:x+w]

        # Detect eyes within the face region
        eyes = eye_cascade.detectMultiScale(face_gray, scaleFactor=1.1, minNeighbors=2, minSize=(30, 30))
        
        for (ex, ey, ew, eh) in eyes:
            eye = face_color[ey:ey + eh, ex:ex + ew]
            eye = cv2.resize(eye, (80, 80)) / 255.0
            eye = eye.reshape(1, 80, 80, 3)

            # Make predictions using the model
            prediction = model.predict(eye)

            # If eyes are closed
            if prediction[0][0] > thresh_closed:
                cv2.putText(frame, 'closed', (10, height - 10), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                            fontScale=0.8, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
                Score += 1
                if Score > 5:
                    sound.play(-1)  # Continuous play if eyes are closed
            # If eyes are open
            elif prediction[0][1] > thresh_open:
                cv2.putText(frame, 'open', (10, height - 10), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                            fontScale=0.8, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
                Score -= 1
                if Score <= 0:
                    Score = 0
                    sound.stop()  # Stop the sound if eyes are open

        # Check head movement (based on face position)
        current_head_position = (x, y)
        if last_head_position is None:
            last_head_position = current_head_position
            head_movement_timer = time.time()  # Start the timer

        # Check if the head position has moved significantly
        head_movement_threshold = 20  # Define how much movement is considered significant
        if abs(current_head_position[0] - last_head_position[0]) < head_movement_threshold and \
           abs(current_head_position[1] - last_head_position[1]) < head_movement_threshold:
            # No significant movement detected
            if time.time() - head_movement_timer >= no_movement_duration:
                cv2.putText(frame, 'HIGHWAY HYPNOSIS', (10, height - 40), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                            fontScale=0.8, color=(0, 0, 255), thickness=1, lineType=cv2.LINE_AA)
                sound.play(-1)  # Play the alarm if no movement for 15 seconds
        else:
            # Reset the timer if movement is detected
            head_movement_timer = time.time()
            sound.stop()  # Stop the alarm
            last_head_position = current_head_position

    # Display the score and text
    cv2.putText(frame, f'Score: {Score}', (70, height - 10), 
                fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=0.8, 
                color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)

    # Display the frame
    cv2.imshow('Drowsiness Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(33) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


2024-10-25 16:38:37.528116: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 428ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━


KeyboardInterrupt

